<a href="https://colab.research.google.com/github/aryansh007/recommenderSystemsForMobileUsers/blob/main/Implementation_KPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving googleplaystore_user_reviews.csv to googleplaystore_user_reviews.csv


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import dateutil.parser as dparser
from sklearn.cluster import KMeans
 
def del_duplicates(df):
  df = df.drop_duplicates(keep='last', inplace=False)
  app_data_df['Last Updated'] = pd.to_datetime(app_data_df['Last Updated'])
  df = df.sort_values(by=['App',  'Last Updated'], ascending=False)
  df = df.drop_duplicates(subset='App', keep='first', inplace=False)
  return df
 
#read dataset
app_data_df = pd.read_csv('googleplaystore.csv')
 
#Drop the abnormal row
app_data_df.drop(app_data_df[app_data_df['App'] == 'Life Made WI-Fi Touchscreen Photo Frame'].index, inplace=True)
 
#Delete all duplicate apps
app_data_df = del_duplicates(app_data_df)
 
#Drop the meaningless attributes
app_data_df = app_data_df[['App', 'Category', 'Rating', 'Reviews', 'Installs', 'Type', 'Price', 'Content Rating']]
 
#Drop the rows with missing values
for row in app_data_df[app_data_df.isnull().any(axis=1)].index:
  app_data_df = app_data_df.drop(row)
app_data_df = app_data_df.reset_index(drop=True)
 
display(app_data_df)

,App,Category,Rating,Reviews,Installs,Type,Price,Content Rating
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,"1,000,000+",Free,0,Everyone
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,"10,000+",Free,0,Everyone
2,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,"1,000,000+",Free,0,Everyone
3,💎 I'm rich,LIFESTYLE,3.8,718,"10,000+",Paid,$399.99,Everyone
4,감성학원 BL 첫사랑,COMICS,4.4,190,"10,000+",Free,0,Everyone
...,...,...,...,...,...,...,...,...
8190,058.ba,NEWS_AND_MAGAZINES,4.4,27,100+,Free,0,Everyone
8191,/u/app,COMMUNICATION,4.7,573,"10,000+",Free,0,Mature 17+
8192,.R,TOOLS,4.5,259,"10,000+",Free,0,Everyone
8193,- Free Comics - Comic Apps,COMICS,3.5,115,"10,000+",Free,0,Mature 17+


In [ ]:
app_org_data_df = app_data_df.copy()
app_data_df = app_data_df[['Category', 'Rating', 'Reviews', 'Installs', 'Type', 'Price', 'Content Rating']]

app_data_df['Installs'] = app_data_df.Installs.str.replace('+', '')
app_data_df['Installs'] = app_data_df.Installs.str.replace(',', '')
app_data_df['Price'] = app_data_df.Price.str.replace('$','')

display(app_data_df)

,Category,Rating,Reviews,Installs,Type,Price,Content Rating
0,ENTERTAINMENT,4.7,11661,1000000,Free,0,Everyone
1,TOOLS,4.0,19,10000,Free,0,Everyone
2,SOCIAL,4.6,22098,1000000,Free,0,Everyone
3,LIFESTYLE,3.8,718,10000,Paid,399.99,Everyone
4,COMICS,4.4,190,10000,Free,0,Everyone
...,...,...,...,...,...,...,...
8190,NEWS_AND_MAGAZINES,4.4,27,100,Free,0,Everyone
8191,COMMUNICATION,4.7,573,10000,Free,0,Mature 17+
8192,TOOLS,4.5,259,10000,Free,0,Everyone
8193,COMICS,3.5,115,10000,Free,0,Mature 17+


In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()

scaler.fit(app_data_df[['Rating']])
app_data_df['Rating'] = scaler.transform(app_data_df[['Rating']])

scaler.fit(app_data_df[['Price']])
app_data_df['Price'] = scaler.transform(app_data_df[['Price']]) 

scaler.fit(app_data_df[['Installs']])
app_data_df['Installs'] = scaler.transform(app_data_df[['Installs']]) 

scaler.fit(app_data_df[['Reviews']])
app_data_df['Reviews'] = scaler.transform(app_data_df[['Reviews']])

display(app_data_df)

,Category,Rating,Reviews,Installs,Type,Price,Content Rating
0,ENTERTAINMENT,0.925,1.491844e-04,9.999990e-04,Free,0.000000,Everyone
1,TOOLS,0.750,2.303018e-07,9.999000e-06,Free,0.000000,Everyone
2,SOCIAL,0.900,2.827211e-04,9.999990e-04,Free,0.000000,Everyone
3,LIFESTYLE,0.700,9.173689e-06,9.999000e-06,Paid,0.999975,Everyone
4,COMICS,0.850,2.418169e-06,9.999000e-06,Free,0.000000,Everyone
...,...,...,...,...,...,...,...
8190,NEWS_AND_MAGAZINES,0.850,3.326582e-07,9.900000e-08,Free,0.000000,Everyone
8191,COMMUNICATION,0.925,7.318480e-06,9.999000e-06,Free,0.000000,Mature 17+
8192,TOOLS,0.875,3.300993e-06,9.999000e-06,Free,0.000000,Everyone
8193,COMICS,0.625,1.458578e-06,9.999000e-06,Free,0.000000,Mature 17+


In [ ]:
app_org_data_df_scaled = app_org_data_df.copy()
app_org_data_df_scaled['Rating'] = app_data_df['Rating']
app_org_data_df_scaled['Reviews'] = app_data_df['Reviews']
app_org_data_df_scaled['Installs'] = app_data_df['Installs']
app_org_data_df_scaled['Price'] = app_data_df['Price']

In [ ]:
pip install kmodes

In [ ]:
from kmodes.kprototypes import KPrototypes

kproto = KPrototypes(n_clusters=7, verbose=2, max_iter=20)
clusters = kproto.fit_predict(app_data_df.values, categorical=[0 ,4, 6])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 2911, ncost: 1306.4797834855647
Run: 1, iteration: 2/20, moves: 2142, ncost: 1303.7340056387006
Run: 1, iteration: 3/20, moves: 230, ncost: 1303.5552668629782
Run: 1, iteration: 4/20, moves: 4, ncost: 1303.5550449477178
Run: 1, iteration: 5/20, moves: 1, ncost: 1303.5550392818714
Run: 1, iteration: 6/20, moves: 0, ncost: 1303.5550392818714
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/20, moves: 2781, ncost: 1194.2878613314929
Run: 2, iteration: 2/20, moves: 1391, ncost: 1192.0141892766972
Run: 2, iteration: 3/20, moves: 908, ncost: 1191.057273756989
Run: 2, iteration: 4/20, moves: 292, ncost: 1190.8262540911298
Run: 2, iteration: 5/20, moves: 32, ncost: 1190.8213844485215
Run: 2, iteration: 6/20, moves: 0, ncost: 1190.8213844485215
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 3, it

In [ ]:
print(kproto.cluster_centroids_)
app_df_labels = app_data_df.copy()
app_df_labels.insert(7, 'Label', clusters)
display(app_df_labels)

app_clusters = {}
for i in range(7):
  app_clusters[i] = app_org_data_df_scaled.iloc[app_df_labels[app_df_labels['Label'] == i].index]

[array([[ 7.86146993e-01,  8.81677183e-04,  2.74728519e-03,
        -3.54594478e-18],
       [ 7.40162037e-01,  1.31212410e-03,  3.51630560e-03,
         9.12037037e-05],
       [ 8.31700288e-01,  1.24933775e-02,  2.87108866e-02,
         4.34942363e-04],
       [ 8.24300699e-01,  4.79070866e-03,  2.08790823e-02,
         4.79676573e-05],
       [ 8.88945087e-01,  4.00178781e-03,  8.49988406e-03,
         1.54927746e-04],
       [ 6.52731092e-01,  2.16617181e-03,  6.23199271e-03,
         1.53378852e-04],
       [ 8.16522989e-01,  1.25200242e-04,  7.17383104e-05,
         3.89858238e-02]]), array([['FAMILY', 'Free', 'Everyone'],
       ['FAMILY', 'Free', 'Teen'],
       ['GAME', 'Free', 'Teen'],
       ['BUSINESS', 'Free', 'Everyone'],
       ['GAME', 'Free', 'Everyone'],
       ['TOOLS', 'Free', 'Everyone'],
       ['FAMILY', 'Paid', 'Everyone']], dtype='<U8')]


,Category,Rating,Reviews,Installs,Type,Price,Content Rating,Label
0,ENTERTAINMENT,0.925,1.491844e-04,9.999990e-04,Free,0.000000,Everyone,4
1,TOOLS,0.750,2.303018e-07,9.999000e-06,Free,0.000000,Everyone,5
2,SOCIAL,0.900,2.827211e-04,9.999990e-04,Free,0.000000,Everyone,4
3,LIFESTYLE,0.700,9.173689e-06,9.999000e-06,Paid,0.999975,Everyone,6
4,COMICS,0.850,2.418169e-06,9.999000e-06,Free,0.000000,Everyone,3
...,...,...,...,...,...,...,...,...
8190,NEWS_AND_MAGAZINES,0.850,3.326582e-07,9.900000e-08,Free,0.000000,Everyone,3
8191,COMMUNICATION,0.925,7.318480e-06,9.999000e-06,Free,0.000000,Mature 17+,4
8192,TOOLS,0.875,3.300993e-06,9.999000e-06,Free,0.000000,Everyone,5
8193,COMICS,0.625,1.458578e-06,9.999000e-06,Free,0.000000,Mature 17+,5


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned


app_review_df = pd.read_csv('/content/googleplaystore_user_reviews.csv')
app_review_df = app_review_df.dropna()
reviews = app_review_df['Translated_Review']
labels = app_review_df['Sentiment']

Stop = set(stopwords.words('english'))
WrdLem = WordNetLemmatizer()
#Cleaning the reviews(removing html tags,punctuation,Lemmatizations) 
nltk.download('wordnet')
Cleaned_sent=[]
for sent in reviews:
    r1=[]
    sent=cleanhtml(sent)
    sent=cleanpunc(sent)
    sent=sent.lower()
    for  word in sent.split():
        if ((word.isalpha()) & (len(word)>2)):
            if word not in Stop:
                w=WrdLem.lemmatize(word)
                r1.append(w)
            else:
                continue
        else:
            continue
    str1 = (" ".join(r1))        
     
    Cleaned_sent.append(str1)

app_review_df['Cleaned_text'] = Cleaned_sent
app_review_df = app_review_df.reset_index(drop=True)
display(app_review_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Cleaned_text
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333,like eat delicious food thats cooking food cas...
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462,help eating healthy exercise regular basis
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000,work great especially going grocery store
3,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000,best idea
4,10 Best Foods for You,Best way,Positive,1.000000,0.300000,best way
...,...,...,...,...,...,...
37422,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667,ad older many agent much owner posted detail r...
37423,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222,photo posted portal load fit purpose sure stor...
37424,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000,dumb app wanted post property rent give option...
37425,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000,property business got link sm happy performanc...


In [ ]:
app_review_copy = app_review_df.copy()
for app in app_review_df['App']:
  if app not in list(app_org_data_df['App']):
    app_review_copy.drop(app_review_copy[app_review_copy['App'] == app].index, inplace=True)
    app_review_copy = app_review_copy.reset_index(drop=True)
app_review_df = app_review_copy
display(app_review_copy)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Cleaned_text
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333,like eat delicious food thats cooking food cas...
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462,help eating healthy exercise regular basis
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000,work great especially going grocery store
3,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000,best idea
4,10 Best Foods for You,Best way,Positive,1.000000,0.300000,best way
...,...,...,...,...,...,...
35885,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667,ad older many agent much owner posted detail r...
35886,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222,photo posted portal load fit purpose sure stor...
35887,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000,dumb app wanted post property rent give option...
35888,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000,property business got link sm happy performanc...


In [ ]:
def filter_ratings(df, app):
  rate_app = df.loc[df['App'] == app].values[0][2]
  return df.loc[df['Rating'] >= rate_app]

def filter_category(df, app):
  cat_app = df.loc[df['App'] == app].values[0][1]
  return df.loc[df['Category'] == cat_app]

def filter_popularity(df, app):
  tmp = df.loc[df['App'] == app].values
  pop_app = tmp[0][2] + tmp[0][3] + tmp[0][4]
  return df.loc[df['Rating'] + df['Reviews'] + df['Installs'] >= pop_app]

a, b = 0, 0
X = app_review_df.values
recommended_apps = [[] for i in range(len(X))]
recommended_apps_df = [[] for i in range(len(X))]
for user_id in range(len(X)):
  label = clusters[app_org_data_df[app_org_data_df['App'] == X[user_id][0]].index][0]
  if X[user_id][2] == 'Negative':
    rec_set = filter_category(filter_ratings(app_clusters[label], X[user_id][0]), X[user_id][0])
    recommended_apps[user_id].append(rec_set.values[:,0])
    recommended_apps_df[user_id] = rec_set.reset_index(drop=True)
    a += 1
  else:
    rec_set = filter_popularity(app_clusters[label], X[user_id][0])
    recommended_apps[user_id].append(rec_set.values[:,0])
    recommended_apps_df[user_id] = rec_set.reset_index(drop=True)
    b += 1
print(a, b)

8001 27889


In [ ]:
cov_set = set()
for i in range(len(X)):
  for j in range(len(recommended_apps[i][0])):
    cov_set.add(recommended_apps[i][0][j])
print(len(cov_set))

7614


In [ ]:
def cal_sim(x, y):
  dissim = (x[1] - y[1]) + (x[2] - y[2]) + (x[3] - y[3]) + (x[5] - y[5])
  if x[0] != y[0]:
    dissim += 1
  if x[4] != y[4]:
    dissim += 1
  if x[6] != y[6]:
    dissim += 1
  dissim /= 7
  return 1 - dissim

sim_mat = np.zeros((len(app_data_df), len(app_data_df)))
for i in range(len(app_data_df)):
  for j in range(len(app_data_df)):
    if i > j:
      break
    sim_mat[i][j] = cal_sim(app_data_df.loc[i], app_data_df.loc[j])
    sim_mat[j][i] = sim_mat[i][j]
sim_mat_df = pd.DataFrame(sim_mat, index=app_org_data_df['App'], columns=app_org_data_df['App'])

In [ ]:
ils = np.zeros(len(X))
for i in range(len(X)):
  if len(recommended_apps[i][0]) <= 1:
    continue
  ils_mat = np.zeros((len(recommended_apps[i][0]), len(recommended_apps[i][0])))
  for j in range(len(recommended_apps[i][0])):
    for k in range(len(recommended_apps[i][0])):
      ils_mat[j][k] = sim_mat_df[recommended_apps[i][0][j]][recommended_apps[i][0][k]]
  ils[i] = np.sum(ils_mat) / (len(recommended_apps[i][0])*len(recommended_apps[i][0]) - len(recommended_apps[i][0]))
avg_ils = np.sum(ils) / len(X)
print(avg_ils)

0.146597215
